# Subject-Driven Generation Metrics Evaluation

##### We have a dataset of 30 distinct “subjects” (objects vs. live subjects/pets), split into:
##### - **Real images**: stored under `data/<subject_name>/…`
##### - **Generated images**: stored under `results/{non-ppl,ppl}/<subject_name>/…`

##### We also have:
##### - `data/subjects.csv` with columns:
#####     - `subject_name` (matches each folder name)
#####    - `class`        (e.g. “dog”, “backpack”, etc.)
#####     - `live`         (boolean: True for pets, False for objects)
##### - `data/prompts.csv` with columns:
#####     - `prompt` (templates containing `{0}` → “sks”, `{1}` → the `class` value)
#####     - `live`   (boolean: whether this prompt applies to live subjects or objects)


##### **Evaluation protocol**:
##### - We generated up to **2** samples per prompt in `ppl` and also **2** for `non-ppl`.
##### - Metrics:
#####     1. **PRES** (avg pairwise DINO similarity real↔gen)
#####     2. **DIV**  (avg pairwise LPIPS distance among gen images)
#####     3. **CLIP-I** (avg cosine between CLIP image embeddings real↔gen)
#####     4. **CLIP-T** (avg cosine between CLIP text embeddings vs. gen images)


In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from metrics import clip_embeddings, div, pres

In [2]:
subjects_df = pd.read_csv('../data/subjects.csv')
prompts_df  = pd.read_csv('../data/prompts.csv')

REAL_ROOT  = '../data'
GEN_ROOT  = '../results'
CONDITIONS = ['no_ppl', 'ppl']

#### Collecting CLIP-I, CLIP-T, PRES (between real and generated images)

In [3]:
results = []
for cond in CONDITIONS:
    data_root = os.path.join(REAL_ROOT, cond)
    res_root = os.path.join(GEN_ROOT, cond)
    for _, row in subjects_df.iterrows():
        subject = row['subject_name']

        real_dir = os.path.join(data_root, subject)
        gen_dir  = os.path.join(res_root, subject)
        if not os.path.isdir(real_dir) or not os.path.isdir(gen_dir):
            continue 

        preservation   = pres.collect_pres(real_dir, gen_dir)
        clip_i, clip_t = None, None

        results.append({
            'condition': cond,
            'subject':   subject,
            'PRES':      preservation,
            'CLIP-I':    clip_i,
            'CLIP-T':    clip_t
        })

        print(f'{cond} {subject} {preservation:.4f}')

results_df = pd.DataFrame(results)

Using cache found in /Users/ignazioperez/.cache/torch/hub/facebookresearch_dino_main


no_ppl backpack 0.6328


Using cache found in /Users/ignazioperez/.cache/torch/hub/facebookresearch_dino_main


no_ppl backpack_dog 0.6952


Using cache found in /Users/ignazioperez/.cache/torch/hub/facebookresearch_dino_main


no_ppl bear_plushie 0.8157


Using cache found in /Users/ignazioperez/.cache/torch/hub/facebookresearch_dino_main


KeyboardInterrupt: 

In [ ]:
results_df.to_csv("metric_results.csv", index=False)

In [ ]:
merged = (
    results_df
    .merge(subjects_df, left_on='subject', right_on='subject_name')
    .drop(columns=['subject_name']) 
)

pivot = (
    merged
    .pivot_table(
       index='class',
       columns='condition',
       values=['PRES','DIV','CLIP-I','CLIP-T'],
       aggfunc='mean'
    )
)
print(pivot)

pivot.to_csv("average_metric_results.csv", index=False)

#### Collecting DIV (between generated images of the same class)

In [15]:
def split_prompts(gen_dir):
    files = [f for f in os.listdir(gen_dir) if f.endswith('.png')]
    groups = defaultdict(list)
    for fn in files:
        parts = fn.split('_')
        p_idx = parts[0]
        groups[p_idx].append(os.path.join(gen_dir, fn))

    return groups

In [16]:
def all_divs(groups):
  prompt_divs = []
  for p_idx, paths in groups.items():
      if len(paths) < 2:
          print(f" only {len(paths)} sample(s) for prompt {p_idx}, skipping")
          continue            
      div_value = div.collect_div(paths)
      prompt_divs.append(div_value)
      #print(f"{cond}/{subject} prompt {p_idx}: DIV = {div_value:.4f}")

  return prompt_divs


In [17]:
div_values = []
for cond in CONDITIONS:
    for _, row in subjects_df.iterrows():
        subject = row['subject_name']
        gen_dir = os.path.join(GEN_ROOT, cond, subject)
        if not os.path.isdir(gen_dir):
            print(f"Skipping {cond}/{subject}: directory not found")
            continue

        groups = split_prompts(gen_dir)    
        prompt_divs = all_divs(groups)

        if not prompt_divs:
            print(f"  No valid prompts for {cond}/{subject}, skipping")
            continue

        subject_div = float(np.mean(prompt_divs))
        div_values.append({
            'condition': cond,
            'subject':   subject,
            'DIV':       subject_div
        })


div_df = pd.DataFrame(div_values)
div_df.to_csv("../results/div_results.csv", index=False)

div_df

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/ignazioperez/opt/anaconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/ignazioperez/opt/anaconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/ignazioperez/opt/anaconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/ignazioperez/opt/anaconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/ignazioperez/opt/anaconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model

,condition,subject,DIV
0,no_ppl,backpack,0.640400
1,no_ppl,backpack_dog,0.473875
2,no_ppl,bear_plushie,0.501589
3,no_ppl,berry_bowl,0.621451
4,no_ppl,can,0.545897
5,no_ppl,candle,0.532296
6,no_ppl,cat,0.463514
7,no_ppl,cat2,0.616232
8,no_ppl,clock,0.615319
9,no_ppl,colorful_sneaker,0.437059


In [18]:
mean_div = (
    div_df
    .groupby('condition')['DIV']
    .mean()
    .reset_index(name='mean_DIV')
)
mean_div

,condition,mean_DIV
0,no_ppl,0.578349
1,ppl,0.635820
